# 한글-Claude-v2 Model: Text summarization with small files with Anthropic Claude

---

### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

## 개요

이 예에서는 소량의 데이터(문자열 데이터)를 Amazon Bedrock API(Anthropic Claude 모델 사용)로 직접 수집하고 각 텍스트를 요약하라는 지침을 제공합니다.

### 건축학

![](./images/41-text-simple-1.png)

이 아키텍처에서:

1. 작은 텍스트(또는 작은 파일)가 로드됨
1. 입력 데이터를 처리하는 기본 모델
1. 모델은 수집된 텍스트의 요약과 함께 응답을 반환합니다.

### 사용 사례

이 접근 방식은 통화 기록, 회의 기록, 서적, 기사, 블로그 게시물 및 기타 관련 콘텐츠를 요약하는 데 사용할 수 있습니다.

### 도전

이 접근 방식은 입력 텍스트 또는 파일이 모델 컨텍스트 길이에 맞을 때 사용할 수 있습니다. 노트북 `02.long-text-summarization-titan.ipynb`에서는 사용자가 토큰 한도를 초과하는 대용량 문서를 가지고 있을 때 문제를 해결하기 위한 접근 방식을 탐색합니다.

# 1. Bedrock Client 생성

In [28]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


# 2. Bedrock Cladue-v2 모델 가져오기

In [29]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# - create the Anthropic Model
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 4096,
        "temperature": 0.5,
        "top_k": 250,
        "top_p": 1,
        "stop_sequences": ["\n\nHuman"],        
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f42fc3228c0>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 4096, 'temperature': 0.5, 'top_k': 250, 'top_p': 1, 'stop_sequences': ['\n\nHuman']}, streaming=True, callbacks=[<langchain.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f42fd014e20>])

# 3. 짧은 텍스트 요약하기
 
아래는 뉴스의 내용을 짧게 요약하는 것을 보여주고 있습니다.

## 뉴스 텍스트 가져오기

In [34]:
# Ref: https://zdnet.co.kr/view/?no=20231013083253
text = """앤서링 AI(Answering AI) 스타트업 포티투마루(42Maru, 대표 김동환)는 생성형 AI 스타트업을 집중 육성하고 지원하기 위한 AWS 프로그램인 'ML Elevate – Generative AI Edition'에 선정됐다고 13일 밝혔다.

이 프로그램은 생성형 AI 스타트업을 지원하기 위한 AWS의 엑셀러레이팅 프로그램이다. 강력한 AI 모델 및 도구, 업계 전문가들의 비즈니스 및 기술 멘토링, 선별된 자원, AWS 액티베이트 프로그램 참여 기회 및 최대 20만 달러 상당의 AWS 크레딧을 제공한다.

주요 혜택은 ▲라이브 온라인 세션, 패널 토론 및 마스터 클래스를 포함한 4주간의 멘토링 ▲135명 이상의 글로벌 ML 설립자 및 운영자로 구성된 Peer to Peer 학습 커뮤니티 ▲AWS 크레딧을 포함한 엄선된 혜택 ▲Accel, KB Investment, Saehan Ventures 등 리딩 VC의 멘토링과 잠재 고객과 온라인 큐레이트 세션 ▲싱가포르에서 열리는 랩업(Wrap-up) 네트워킹 행사 참가 등이다.
앞서 포티투마루는 지난 5월 환각(Hallucination), 보안, 고비용 문제를 해결한 기업용 거대언어모델(LLM)인 ‘LLM42’를 공개한 바 있다. 경량화 모델 구현을 통해 솔루션 구축과 운영에 들어가는 비용을 획기적으로 절감하고 ‘프라이빗 모드’를 지원해 기업 및 고객 정보에 대한 보안 문제를 해결한 것이 특징이다. 또 지난 9월에는 방대한 양의 대규모 원천 데이터로부터 LLM 모델이 필요로 하는 최적의 정보나 지식만 도출하고 정제하는 기술인 ‘RAG42’를 출시, 모델 종류에 상관없이 각 산업군별 도메인 특화 엔진 개발이 가능하도록 학습데이터 구축부터 파인 튜닝까지 전과정을 지원하고 있다.

포티투마루는 초거대 AI 관련 핵심 기술을 차례로 공개하며 컨설팅부터 솔루션 구축까지 원스톱 서비스를 제공함으로써 엔터프라이즈 LLM 분야의 저변을 확대하는 등 글로벌 생성형 AI 시장에서 빠른 행보를 이어가고 있다
김동환 포티투마루 대표는 "엄선된 생성형 AI 스타트업을 대상으로 진행하는 프로그램에 선정돼 참여중"이라면서 "상당히 수준 높고 전문적인 프로그램으로 생성형 AI 생태계의 중심에 와 있는 느낌이 든다"는 소감을 밝혔다. 이어 "AWS 지원과 지속적인 협력을 통해 포티투마루의 LLM42, RAG42 및 사이트버니 솔루션을 아마존 세이지메이커 점프스타트(Amazon SageMaker JumpStart)를 통해 글로벌 SaaS 서비스로 제공할 예정"이라고 덧붙였다."""

In [35]:
print("text: \n", text)

text: 
 앤서링 AI(Answering AI) 스타트업 포티투마루(42Maru, 대표 김동환)는 생성형 AI 스타트업을 집중 육성하고 지원하기 위한 AWS 프로그램인 'ML Elevate – Generative AI Edition'에 선정됐다고 13일 밝혔다.

이 프로그램은 생성형 AI 스타트업을 지원하기 위한 AWS의 엑셀러레이팅 프로그램이다. 강력한 AI 모델 및 도구, 업계 전문가들의 비즈니스 및 기술 멘토링, 선별된 자원, AWS 액티베이트 프로그램 참여 기회 및 최대 20만 달러 상당의 AWS 크레딧을 제공한다.

주요 혜택은 ▲라이브 온라인 세션, 패널 토론 및 마스터 클래스를 포함한 4주간의 멘토링 ▲135명 이상의 글로벌 ML 설립자 및 운영자로 구성된 Peer to Peer 학습 커뮤니티 ▲AWS 크레딧을 포함한 엄선된 혜택 ▲Accel, KB Investment, Saehan Ventures 등 리딩 VC의 멘토링과 잠재 고객과 온라인 큐레이트 세션 ▲싱가포르에서 열리는 랩업(Wrap-up) 네트워킹 행사 참가 등이다.
앞서 포티투마루는 지난 5월 환각(Hallucination), 보안, 고비용 문제를 해결한 기업용 거대언어모델(LLM)인 ‘LLM42’를 공개한 바 있다. 경량화 모델 구현을 통해 솔루션 구축과 운영에 들어가는 비용을 획기적으로 절감하고 ‘프라이빗 모드’를 지원해 기업 및 고객 정보에 대한 보안 문제를 해결한 것이 특징이다. 또 지난 9월에는 방대한 양의 대규모 원천 데이터로부터 LLM 모델이 필요로 하는 최적의 정보나 지식만 도출하고 정제하는 기술인 ‘RAG42’를 출시, 모델 종류에 상관없이 각 산업군별 도메인 특화 엔진 개발이 가능하도록 학습데이터 구축부터 파인 튜닝까지 전과정을 지원하고 있다.

포티투마루는 초거대 AI 관련 핵심 기술을 차례로 공개하며 컨설팅부터 솔루션 구축까지 원스톱 서비스를 제공함으로써 엔터프라이즈 LLM 분야의 저변을 확대하는 등 글로벌 생성형 AI 시장에서 빠른 행보를 이어가고 있다
김동

## Langchain PromptTemplate 를 통하여 prompt 생성

In [36]:
from langchain import PromptTemplate

summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="""\n\nHuman: 다음 텍스트를 요약 해주세요.
<text>{text}</text>\n\nAssistant:"""
)

prompt = summary_prompt.format(text= text)

prompt

'\n\nHuman: 다음 텍스트를 요약 해주세요.\n<text>앤서링 AI(Answering AI) 스타트업 포티투마루(42Maru, 대표 김동환)는 생성형 AI 스타트업을 집중 육성하고 지원하기 위한 AWS 프로그램인 \'ML Elevate – Generative AI Edition\'에 선정됐다고 13일 밝혔다.\n\n이 프로그램은 생성형 AI 스타트업을 지원하기 위한 AWS의 엑셀러레이팅 프로그램이다. 강력한 AI 모델 및 도구, 업계 전문가들의 비즈니스 및 기술 멘토링, 선별된 자원, AWS 액티베이트 프로그램 참여 기회 및 최대 20만 달러 상당의 AWS 크레딧을 제공한다.\n\n주요 혜택은 ▲라이브 온라인 세션, 패널 토론 및 마스터 클래스를 포함한 4주간의 멘토링 ▲135명 이상의 글로벌 ML 설립자 및 운영자로 구성된 Peer to Peer 학습 커뮤니티 ▲AWS 크레딧을 포함한 엄선된 혜택 ▲Accel, KB Investment, Saehan Ventures 등 리딩 VC의 멘토링과 잠재 고객과 온라인 큐레이트 세션 ▲싱가포르에서 열리는 랩업(Wrap-up) 네트워킹 행사 참가 등이다.\n앞서 포티투마루는 지난 5월 환각(Hallucination), 보안, 고비용 문제를 해결한 기업용 거대언어모델(LLM)인 ‘LLM42’를 공개한 바 있다. 경량화 모델 구현을 통해 솔루션 구축과 운영에 들어가는 비용을 획기적으로 절감하고 ‘프라이빗 모드’를 지원해 기업 및 고객 정보에 대한 보안 문제를 해결한 것이 특징이다. 또 지난 9월에는 방대한 양의 대규모 원천 데이터로부터 LLM 모델이 필요로 하는 최적의 정보나 지식만 도출하고 정제하는 기술인 ‘RAG42’를 출시, 모델 종류에 상관없이 각 산업군별 도메인 특화 엔진 개발이 가능하도록 학습데이터 구축부터 파인 튜닝까지 전과정을 지원하고 있다.\n\n포티투마루는 초거대 AI 관련 핵심 기술을 차례로 공개하며 컨설팅부터 솔루션 구축까지 원스톱 서비스를 제공함으로써 엔터프라이즈 LLM 분야의 저변을 확대하

## 4. LLM (Claude-v2) 로 뉴스 요약

In [37]:
llm(prompt)

 앤서링 AI 스타트업 포티투마루가 AWS의 'ML Elevate - Generative AI Edition' 프로그램에 선정되었다. 이 프로그램은 생성형 AI 스타트업을 지원하기 위한 것으로, 멘토링, AWS 크레딧 등의 혜택을 제공한다. 포티투마루는 생성형 AI 모델 LLM42와 RAG42를 공개한 바 있으며, AWS와의 협력을 통해 이를 글로벌 SaaS 서비스로 제공할 예정이다.

" 앤서링 AI 스타트업 포티투마루가 AWS의 'ML Elevate - Generative AI Edition' 프로그램에 선정되었다. 이 프로그램은 생성형 AI 스타트업을 지원하기 위한 것으로, 멘토링, AWS 크레딧 등의 혜택을 제공한다. 포티투마루는 생성형 AI 모델 LLM42와 RAG42를 공개한 바 있으며, AWS와의 협력을 통해 이를 글로벌 SaaS 서비스로 제공할 예정이다."

## 5. 결론

### Next Action
- 프롬프트를 특정 사용 사례로 변경하고 다른 모델의 출력을 평가합니다.
- 더 나은 결과를 얻기 위해 다양한 프롬프트 엔지니어링 원칙을 적용합니다.